In [2]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.product_utils import *
from manenv.world import World
from manenv.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.product import *
from manenv.effector import *

import manenv.effectors as eff
import manenv.components as cmp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'pygame_gui'

In [ ]:
world = World((3, 3))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 2), cmp.Spawner(product))
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [1, 0, 0],
    [1, 0, -1],
    [1, 0, 0]
])))

grabber = eff.Grabber(make_vector(0, 0))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 4))
discard = eff.Acceptor(make_vector(1, 7))

assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard])
world.place_component(make_vector(1, 0), assembler)
wrapper = RenderWrapper(world)

In [ ]:
assembler.place_in_inventory(product.copy())
world.update()

grabber.set_action(eff.GrabberActions.GRAB_INVENTORY)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE)
world.update()

grabber.set_action(eff.GrabberActions.GRAB)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.ROTATE_CCW)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE)
world.update()

grabber.set_action(eff.GrabberActions.GRAB)
buddy.set_action(eff.GrabberActions.GRAB)
world.update()

buddy.set_action(eff.GrabberActions.MOVE_LEFT)
grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

buddy.set_action(eff.GrabberActions.RELEASE)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_FORWARD)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE)
world.update()

In [ ]:
wrapper.render(update_mode=True)